In [10]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings("ignore")
from postgresRDSConnect import host, database, user, password, port

In [11]:
# Establishing a connection
try:
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password,
        port=port
    )
    print("Connection to PostgreSQL database successful")
except psycopg2.Error as e:
    print("Error: Could not connect to PostgreSQL database")
    print(e)

Connection to PostgreSQL database successful


In [5]:
query = 'SELECT * FROM "evChargers"."EV_Charger_Extract";'

In [6]:
df = pd.read_sql_query(query, conn)

In [ ]:
df.head()

In [ ]:
select_columns = df[['station_name',
 'street_address',
 'city',
 'state',
 'zip',
 'country',
 'facility_type',
 'latitude',
 'longitude',
 'ev_network']]

select_columns.head()

In [9]:
select_columns.loc[select_columns['state'] == 'NJ', 'zip'] = select_columns.loc[select_columns['state'] == 'NJ', 'zip'].apply(lambda x: str(x).zfill(5))

In [ ]:
filtered_rows = select_columns[select_columns['state'] == 'NJ']
filtered_rows.head()

In [ ]:
ev_charging_updated_df = select_columns
ev_charging_updated_df.head()

In [9]:
ev_charging_updated_df.dtypes

station_name       object
street_address     object
city               object
state              object
zip                object
country            object
facility_type      object
latitude          float64
longitude         float64
ev_network         object
dtype: object

In [ ]:
ev_charging_updated_df['zip'] = ev_charging_updated_df['zip'].astype(str)
ev_charging_updated_df['city'] = ev_charging_updated_df['city'].astype(str)
ev_charging_updated_df['state'] = ev_charging_updated_df['state'].astype(str)

In [ ]:

# Construct the connection string
db_connection_str = f'postgresql://{user}:{password}@{host}:{port}/{database}'

# Define the schema name and view name
schema_name = 'evChargers'
table_name = 'EV_Charging_Tristate'

# Write the DataFrame as a view to the database
ev_charging_updated_df.to_sql(table_name, con=create_engine(db_connection_str), schema=schema_name, if_exists='replace', index=False)
